<a href="https://colab.research.google.com/github/Smart-Pigs/Depth-Images/blob/main/Weight_Estimation_TransferLearning_Cropped_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure you have this folder as shortcut in your drive - /content/gdrive/MyDrive/Morota-video-data/ 

In [1]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


### Data Loading

In [2]:
#@markdown Select resolution of input dataset and weeks of data

Instantiate_New_Dataset = True #@param {type:"boolean"}
Refresh_Old_Dataset = False #@param {type:"boolean"}

if Instantiate_New_Dataset == True:
  !rm -rf Week*/

Resolution = "256"  #@param ['720', '512', '256']

if Refresh_Old_Dataset == False:
  if Resolution == "720":
    !cp /content/gdrive/MyDrive/SmartPig/References/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_720.zip .
    !unzip -qq CROPPED_WEEK1-10_720.zip -d .
    !rm -rf *WEEK*.zip

  if Resolution == "512":
    !cp /content/gdrive/MyDrive/SmartPig/References/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_512.zip .
    !unzip -qq CROPPED_WEEK1-10_512.zip -d .
    !rm -rf *WEEK*.zip

  if Resolution == "256":
    !cp /content/gdrive/MyDrive/SmartPig/References/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_256.zip .
    !unzip -qq CROPPED_WEEK1-10_256.zip -d .
    !rm -rf *WEEK*.zip

### Mapping Table Input

In [3]:
import pandas as pd

In [4]:
final_map = pd.read_csv("/content/gdrive/MyDrive/SmartPig/References/Morota-video-data/final_mapping.csv")

In [5]:
weeks = final_map

In [6]:
weeks

,image_path,Date,Pig,weight,Week
0,Week1/20210922/20210922_3330_b/_Depth_4683.jpg,20210922,3330,15.5,1
1,Week1/20210922/20210922_3330_b/_Depth_4778.jpg,20210922,3330,15.5,1
2,Week1/20210922/20210922_3330_b/_Depth_4579.jpg,20210922,3330,15.5,1
3,Week1/20210922/20210922_3330_b/_Depth_4578.jpg,20210922,3330,15.5,1
4,Week1/20210922/20210922_3330_b/_Depth_4584.jpg,20210922,3330,15.5,1
...,...,...,...,...,...
96220,Week10/20211119/20211119_3342/_Depth_3504.jpg,20211119,3342,56.6,10
96221,Week10/20211119/20211119_3342/_Depth_3318.jpg,20211119,3342,56.6,10
96222,Week10/20211119/20211119_3342/_Depth_3476.jpg,20211119,3342,56.6,10
96223,Week10/20211119/20211119_3342/_Depth_3169.jpg,20211119,3342,56.6,10


In [7]:
weeks_train = weeks[weeks["Pig"] != 3384]

In [8]:
weeks_test = weeks[weeks["Pig"] == 3384]

In [9]:
image_paths_train = weeks_train['image_path'].values
labels_train = weeks_train['weight'].values
image_paths_test = weeks_test['image_path'].values
labels_test = weeks_test['weight'].values

### Dataset

In [10]:
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import EfficientNetB5

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((image_paths_train, labels_train))
test_dataset = tf.data.Dataset.from_tensor_slices((image_paths_test, labels_test))

In [12]:
IMG_SIZE = int(Resolution)
channels = 3

def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    #img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.rgb_to_grayscale(img)
    if int(Resolution) > IMG_SIZE:
      img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

In [13]:
print("Number of elements in the train dataset:", tf.data.experimental.cardinality(train_dataset).numpy())
print("Number of elements in the test dataset:", tf.data.experimental.cardinality(test_dataset).numpy())

Number of elements in the train dataset: 77871
Number of elements in the test dataset: 18354


In [14]:
batch_size = 128

train_dataset = train_dataset.shuffle(len(labels_train))
train_dataset = train_dataset.map(preprocess_image)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_image)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

### Model Config and Training

In [16]:
from keras.layers.normalization.batch_normalization import BatchNormalization
base_model = EfficientNetB5(weights='imagenet', 
                            input_tensor=None,
                            input_shape=None,
                            include_top = False);
base_model.trainable = False;
for layer in base_model.layers[-18:]:
  if not isinstance(layer, BatchNormalization):
    layer.trainable = True;

115263384/115263384 [==============================] - 4s 0us/step


In [17]:
from keras.layers.pooling.global_average_pooling2d import GlobalAveragePooling2D
def head(base_output):
  # x_input = tf.keras.layers.Input(shape)
  x= GlobalAveragePooling2D()(base_output)
  x= BatchNormalization()(x)

  x= tf.keras.layers.Dense(units = 64, kernel_regularizer= 'l2',kernel_initializer='he_uniform')(x)
  x= BatchNormalization()(x)
  x= tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)

  x= tf.keras.layers.Dense(units = 128, kernel_regularizer= 'l2',kernel_initializer='he_uniform')(x)
  x= BatchNormalization()(x)
  x= tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)

  x = tf.keras.layers.Dense(1)(x);
  # model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
  
  return x


In [22]:
model= tf.keras.models.Model(base_model.input, head(base_model.output), name='transferlearning')

In [24]:
model.compile(optimizer='adam', loss='mse', metrics=['mae','mape'])

In [25]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
609/609 [==============================] - 833s 1s/step - loss: 302.6367 - mae: 13.2568 - mape: 40.7760 - val_loss: 45.7607 - val_mae: 5.4014 - val_mape: 19.2006
Epoch 2/10
609/609 [==============================] - 809s 1s/step - loss: 81.6691 - mae: 6.9239 - mape: 23.1148 - val_loss: 228.4673 - val_mae: 14.4493 - val_mape: 60.8355
Epoch 3/10
609/609 [==============================] - 793s 1s/step - loss: 73.7976 - mae: 6.5448 - mape: 21.9811 - val_loss: 87.6849 - val_mae: 8.5248 - val_mape: 36.4990
Epoch 4/10
609/609 [==============================] - 808s 1s/step - loss: 68.2628 - mae: 6.2697 - mape: 21.1080 - val_loss: 97.1125 - val_mae: 9.2815 - val_mape: 38.2820
Epoch 5/10
609/609 [==============================] - 794s 1s/step - loss: 65.8185 - mae: 6.1430 - mape: 20.6785 - val_loss: 26.4609 - val_mae: 4.0405 - val_mape: 16.5101
Epoch 6/10
609/609 [==============================] - ETA: 0s - loss: 63.7125 - mae: 6.0438 - mape: 20.3502

KeyboardInterrupt: ignored

In [26]:
test_loss, test_mae, test_mape = model.evaluate(test_dataset)
print('Test loss:', test_loss)
print('Test mean absolute error:', test_mae)
print('Test mean absolute percentage error:', test_mape)

144/144 [==============================] - 129s 896ms/step - loss: 59.1268 - mae: 6.8354 - mape: 29.1205
Test loss: 59.12681198120117
Test mean absolute error: 6.835434436798096
Test mean absolute percentage error: 29.120502471923828
